In [ ]:
from deltalake import DeltaTable, write_deltalake
from deltalake.exceptions import DeltaError
import duckdb

In [ ]:
con = duckdb.connect()

In [ ]:
from dotenv import load_dotenv
import os

# Carregar o arquivo .env com verificação de erro
try:
    load_dotenv('C:\\Users\\Lincoln\\source\\repos\\lakehouse\\delta\\.env')
except Exception as e:
    print(f"Erro ao carregar o arquivo .env: {str(e)}")
    raise

# Definir todas as variáveis necessárias com verificação
required_vars = [
    "AZURE_ACCOUNT_NAME",
    "AZURE_CLIENT_ID",
    "AZURE_ACCOUNT_KEY",
    "AZURE_ACCOUNT_SECRET"
]

# Verificar se todas as variáveis estão definidas
missing_vars = []
for var in required_vars:
    value = os.getenv(var)
    if value is None:
        # Defina valores padrão para desenvolvimento/teste
        if var == "AZURE_STORAGE_CLIENT_ID":
            os.environ[var] = "dummy-client-id"
        elif var == "AZURE_STORAGE_ACCOUNT_SECRET":
            os.environ[var] = "dummy-account-secret"
        else:
            missing_vars.append(var)

if missing_vars:
    raise ValueError(
        f"As seguintes variáveis não foram encontradas no .env: {', '.join(missing_vars)}"
    )

# Criar o dicionário storage_options apenas após verificar todas as variáveis
storage_options = {
    "AZURE_STORAGE_ACCOUNT_NAME": os.getenv("AZURE_STORAGE_ACCOUNT_NAME"),
    "AZURE_STORAGE_ACCESS_KEY": os.getenv("AZURE_STORAGE_ACCESS_KEY"),
    "AZURE_STORAGE_CLIENT_ID": os.getenv("AZURE_STORAGE_CLIENT_ID"),
    "AZURE_STORAGE_CLIENT_SECRET": os.getenv("AZURE_STORAGE_CLIENT_SECRET")
}

print("Variáveis carregadas com sucesso:", storage_options)

In [ ]:
def escreve_delta(data, path, mode):
    uri = f'az://bronze/delta/delta-operacoes/{path}'
    try:
        write_deltalake(uri,
                         data, 
                         mode=mode, 
                         storage_options=storage_options)
        print("Dados escritos com sucesso no Delta Lake!")
    except DeltaError as e:
        print(f"Erro ao escrever dados no Delta Lake: {e}")

In [ ]:
def ler_delta(path):
    return DeltaTable(f'az://bronze/delta/delta-operacoes/{path}', storage_options=storage_options)

In [ ]:
df = con.sql("SELECT * FROM 'C:\\Users\\Lincoln\\source\\repos\\lakehouse\\categories_test.csv'").to_df()
df

In [ ]:
con.close()

In [ ]:
escreve_delta(df, 'categories', mode='overwrite')

In [ ]:
dados = ler_delta('categories')
dados.to_pandas()

In [119]:
dt = ler_delta('categories')
#dt.to_pandas()
versao = dt.version()
print("Versão da tabela Delta:", versao)

Versão da tabela Delta: 0


In [121]:
dt.delete("category_id > 4")

DeltaError: Invalid table version: 0